In [1]:
#Colab specific code

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/deep-image-prior-master
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/deep-image-prior-master
/content/drive/My Drive/deep-image-prior-master


In [0]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import argparse
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from skimage.measure import compare_psnr
from models.downsampler import Downsampler

from utils.sr_utils import *

import numpy as np
from models import *

import torch
import torch.optim

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

imsize =-1 
factor = 4
enforse_div32 = 'CROP' 

PLOT = False

In [0]:
#Define imagesets and load

Set5 = {"baby": "img_001_SRF_2_", "bird": "img_002_SRF_2_", "butterfly": "img_003_SRF_2_", "head": 'img_004_SRF_2_', "woman": 'img_005_SRF_2_'}
Set14 = {"baboon": "img_001_SRF_2_", "barbara": "img_002_SRF_2_", "bridge": "img_003_SRF_2_", "coastguard":"img_004_SRF_2_", "comic":"img_005_SRF_2_", "face":"img_006_SRF_2_", "flowers":"img_007_SRF_2_", "foreman":"img_008_SRF_2_", "lenna":"img_009_SRF_2_", "man":"img_010_SRF_2_", "monarch":"img_011_SRF_2_", "pepper":"img_012_SRF_2_", "ppt3":"img_013_SRF_2_", "zebra": "img_014_SRF_2_"}

set_5 = "Set5/"
set_14 ="Set14/"


In [0]:
def process_output(result_deep_prior, psnr_history_deep_prior, IMAGE):
    print("Last PSNR: ", psnr_history_deep_prior)
    print(result_deep_prior.shape)
    plt.imshow(np.clip(result_deep_prior.transpose(1,2,0), 0, 1))
    outfile = "loopoutput/SR_deep_prior_SKIP_" + str(IMAGE) + ".png"
    plt.imsave(outfile, (np.clip(result_deep_prior.transpose(1,2,0), 0, 1)))
    #Write the PSNR here or return them to take some averages

In [0]:
#fname -- The filename from the imageset (ie, 5 or 14)
#IMAGE, the name of the image, (ie, bird, woman)
def process_image(fname, IMAGE):
    imgs = load_LR_HR_imgs_sr(fname, imsize, factor, enforse_div32)

    if PLOT:
        imgs['bicubic_np'], imgs['sharp_np'], imgs['nearest_np'] = get_baselines(imgs['LR_pil'], imgs['HR_pil'])
        plot_image_grid([imgs['HR_np'], imgs['bicubic_np'], imgs['sharp_np'], imgs['nearest_np']], 4,12);
        print ('PSNR bicubic: %.4f   PSNR nearest: %.4f' %  (
                                            compare_psnr(imgs['HR_np'], imgs['bicubic_np']), 
                                            compare_psnr(imgs['HR_np'], imgs['nearest_np'])))
        
    i = 0
    def closure():
        
        nonlocal i, net_input
        
        
        if reg_noise_std > 0:
            net_input = net_input_saved + (noise.normal_() * reg_noise_std)
            
        out_HR = net(net_input)
        out_LR = downsampler(out_HR)

        total_loss = mse(out_LR, img_LR_var) + tv_weight * tv_loss(out_HR)
        total_loss.backward()

        # Log
        psnr_LR = compare_psnr(imgs['LR_np'], torch_to_np(out_LR))
        psnr_HR = compare_psnr(imgs['HR_np'], torch_to_np(out_HR))
        print ('Iteration %05d    PSNR_LR %.3f   PSNR_HR %.3f' % (i, psnr_LR, psnr_HR), '\r', end='')
                          
        # History
        psnr_history.append([psnr_LR, psnr_HR])
        
        if PLOT and i % 500 == 0:
            out_HR_np = torch_to_np(out_HR)
            plot_image_grid([imgs['HR_np'], np.clip(out_HR_np, 0, 1)], factor=8, nrow=2, interpolation='lanczos')

        i += 1
        
        return total_loss

    input_depth = 3
     
    INPUT = 'noise'
    pad = 'reflection'
    OPT_OVER = 'input'
    KERNEL_TYPE='lanczos2'

    LR = 0.01
    tv_weight = 0.0

    OPTIMIZER = 'adam'

    num_iter = 2000
    reg_noise_std = 0.0

    # Identity mapping network, optimize over `net_input`
    net = nn.Sequential()
    net_input = get_noise(input_depth, INPUT, (imgs['HR_pil'].size[1], imgs['HR_pil'].size[0])).type(dtype).detach()

    downsampler = Downsampler(n_planes=3, factor=factor, kernel_type='lanczos2', phase=0.5, preserve_size=True).type(dtype)

    # Loss
    mse = torch.nn.MSELoss().type(dtype)

    img_LR_var = np_to_torch(imgs['LR_np']).type(dtype)

    psnr_history = [] 
    net_input_saved = net_input.detach().clone()
    noise = net_input.detach().clone()
     
    p = get_params(OPT_OVER, net, net_input)
    optimize(OPTIMIZER, p, closure, LR, num_iter)

    out_HR_np = np.clip(torch_to_np(net(net_input)), 0, 1)

    result_no_prior = put_in_center(out_HR_np, imgs['orig_np'].shape[1:])
    psnr_history_direct = psnr_history

    net_input = get_noise(input_depth, INPUT, (imgs['HR_pil'].size[1], imgs['HR_pil'].size[0])).type(dtype).detach()

    psnr_history = [] 
     
    p = get_params(OPT_OVER, net, net_input)
    optimize(OPTIMIZER, p, closure, LR, num_iter)
    out_HR_np = np.clip(torch_to_np(net(net_input)), 0, 1)

    result_tv_prior = put_in_center(out_HR_np, imgs['orig_np'].shape[1:])
    psnr_history_tv = psnr_history

    #Deep prior part
    OPT_OVER = 'net'
    reg_noise_std = 1./30. 
    tv_weight = 0.0

    net = skip().type(torch.cuda.FloatTensor)

    net_input = get_noise(input_depth, INPUT, (imgs['HR_pil'].size[1], imgs['HR_pil'].size[0])).type(dtype).detach()

    # Compute number of parameters
    s  = sum([np.prod(list(p.size())) for p in net.parameters()]); 
    print ('Number of params: %d' % s)

    psnr_history = [] 
    net_input_saved = net_input.detach().clone()
    noise = net_input.detach().clone()

    i = 0
    p = get_params(OPT_OVER, net, net_input)
    optimize(OPTIMIZER, p, closure, LR, num_iter)

    out_HR_np = np.clip(torch_to_np(net(net_input)), 0, 1)

    result_deep_prior = put_in_center(out_HR_np, imgs['orig_np'].shape[1:])
    psnr_history_deep_prior = psnr_history
    process_output(result_deep_prior, psnr_history_deep_prior, IMAGE)


In [0]:
  #Loop through all images in a set
  #for IMAGE in Set5:
  #    fname  = set_5 + Set5[IMAGE] + "HR" + ".png"
  #    process_image(fname, IMAGE)
  #    net.summary()

  #Compare two of them
  #Butterfly
  fname  = set_5 + Set5["butterfly"] + "HR" + ".png"
  process_image(fname, "butterfly")

  #Barbara
  fname  = set_14 + Set14["barbara"] + "HR" + ".png"
  process_image(fname, "barbara")

HR and LR resolutions: (256, 256), (64, 64)
Starting optimization with ADAM
Starting optimization with ADAM
